# Notebook 31 — Model Inference: Predicting Opening Performance

## Purpose

This notebook performs **pure inference** on processed player data using the trained chess opening recommender model.

**Input**: Processed `ModelInput` object from notebook 29 (or from production API)

**Output**: Ranked opening recommendations with predicted performance scores

**Key Design Principles**:
- **Data ↔ Behavior Separation**: Load data first, then define pure functions, then apply functions to data
- **Granular Functions**: Each step is a small, testable, reusable function
- **Production-Ready**: Code can be directly adapted for HuggingFace Spaces deployment
- **Fold-in Users**: Handles new players (not in training set) using rating and opening features only

---

## Pipeline Overview

```
1. SETUP & CONFIGURATION
   ├── Import libraries
   ├── Define paths and specs
   └── Set random seeds

2. LOAD DATA
   ├── Load processed player data (ModelInput)
   ├── Load model artifacts (hyperparameters, mappings, etc.)
   └── Load opening metadata for display

3. DEFINE MODEL ARCHITECTURE
   ├── ChessOpeningRecommender class (from training)
   └── Helper methods for embeddings

4. LOAD TRAINED MODEL
   ├── Instantiate model with correct hyperparameters
   ├── Load trained weights (best_model.pt)
   └── Set to evaluation mode

5. DEFINE INFERENCE FUNCTIONS
   ├── convert_to_tensors() - ModelInput → PyTorch tensors
   ├── generate_predictions() - Run model forward pass
   ├── predict_all_openings() - Get scores for ALL valid openings
   └── rank_recommendations() - Sort and filter results

6. RUN INFERENCE
   ├── Convert player data to tensors
   ├── Generate predictions
   ├── Predict scores for all openings (including unplayed)
   └── Rank and display recommendations

7. ANALYZE RESULTS
   ├── Compare predictions vs actuals (for played openings)
   ├── Calculate error metrics
   ├── Display top recommendations
   └── Optional: Visualizations
```

---

## Notes

- This notebook **never modifies model weights** - purely inference
- Designed for easy HuggingFace Spaces deployment (~2-4 seconds per user)
- Handles fold-in users (new players not in training set)
- All functions are stateless and pure (no side effects)

---

# PART 1: SETUP & CONFIGURATION

In [1]:
# Standard library imports
import sys
from pathlib import Path
import json
import pickle
from typing import Dict, List, Tuple, Optional

# Third-party imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

# Add project root to path for module imports
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from utils.foldin_data_processing.types import ModelInput

print("✓ All imports successful")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/a/Documents/personalprojects/chess-opening-recommender/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/a/Documents/personalprojects/chess-opening-recommender/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/a/Documents/personalprojects/chess-opening-

✓ All imports successful


In [2]:
# ============================================================================
# INFERENCE SPECIFICATIONS - Adjust these parameters as needed
# ============================================================================

# Color to analyze ('w' for White, 'b' for Black)
COLOR = 'b'

# Model artifacts directory name
MODEL_DIR_NAME = "20251212_152017_black"

# Number of recommendations to display
TOP_N_RECOMMENDATIONS = 10

# Random seed for reproducibility
RANDOM_SEED = 42

# Device for inference (CPU is fine for this model size)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ============================================================================

# Derived display values
COLOR_NAME = "White" if COLOR == 'w' else "Black"

print("═" * 80)
print("INFERENCE SPECIFICATIONS")
print("═" * 80)
print(f"Color: {COLOR_NAME} ('{COLOR}')")
print(f"Model directory: {MODEL_DIR_NAME}")
print(f"Top N recommendations: {TOP_N_RECOMMENDATIONS}")
print(f"Device: {DEVICE}")
print(f"Random seed: {RANDOM_SEED}")
print("═" * 80)

════════════════════════════════════════════════════════════════════════════════
INFERENCE SPECIFICATIONS
════════════════════════════════════════════════════════════════════════════════
Color: Black ('b')
Model directory: 20251212_152017_black
Top N recommendations: 10
Device: cpu
Random seed: 42
════════════════════════════════════════════════════════════════════════════════


In [3]:
# Set random seeds for reproducibility
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

print("✓ Random seeds set")

✓ Random seeds set


In [4]:
# Configuration - construct paths from specifications
DATA_DIR = Path.cwd() / "data"
MODEL_ARTIFACTS_DIR = PROJECT_ROOT / "data" / "models" / MODEL_DIR_NAME
PLAYER_DATA_PATH = DATA_DIR / f"processed_player_data_{COLOR}.pkl"

# Verify paths exist
print("Verifying paths...")
print(f"   Model artifacts: {MODEL_ARTIFACTS_DIR.exists()} - {MODEL_ARTIFACTS_DIR}")
print(f"   Player data: {PLAYER_DATA_PATH.exists()} - {PLAYER_DATA_PATH}")

if not MODEL_ARTIFACTS_DIR.exists():
    raise FileNotFoundError(f"Model artifacts not found at {MODEL_ARTIFACTS_DIR}")
if not PLAYER_DATA_PATH.exists():
    raise FileNotFoundError(
        f"Processed player data not found at {PLAYER_DATA_PATH}\n"
        f"Run notebook 29 first to generate this file."
    )

print("✓ All paths verified")

Verifying paths...
   Model artifacts: True - /Users/a/Documents/personalprojects/chess-opening-recommender/data/models/20251212_152017_black
   Player data: True - /Users/a/Documents/personalprojects/chess-opening-recommender/notebooks/data/processed_player_data_b.pkl
✓ All paths verified


---

# PART 2: LOAD DATA

Load all necessary data:
- Processed player data (ModelInput)
- Model artifacts (hyperparameters, mappings)
- Opening metadata for display

In [5]:
# Load processed player data from notebook 29
print("Loading processed player data...")

with open(PLAYER_DATA_PATH, 'rb') as f:
    model_input: ModelInput = pickle.load(f)

print(f"✓ Loaded player data")
print(f"   Player ID: {model_input.training_player_id} (None = fold-in user)")
print(f"   Rating Z: {model_input.rating_z:.4f}")
print(f"   Number of openings played: {len(model_input)}")
print(f"   Opening IDs shape: {model_input.opening_ids.shape}")
print(f"   ECO letter cats shape: {model_input.eco_letter_cats.shape}")
print(f"   Scores shape: {model_input.scores.shape}")

Loading processed player data...
✓ Loaded player data
   Player ID: None (None = fold-in user)
   Rating Z: -0.1696
   Number of openings played: 94
   Opening IDs shape: (94,)
   ECO letter cats shape: (94,)
   Scores shape: (94,)


In [6]:
# Load model hyperparameters
print("\nLoading model hyperparameters...")

hyperparams_path = MODEL_ARTIFACTS_DIR / "hyperparameters.json"
with open(hyperparams_path, 'r') as f:
    hyperparams = json.load(f)

# Extract key hyperparameters
NUM_PLAYERS = hyperparams['num_players']
NUM_OPENINGS = hyperparams['num_openings']
NUM_FACTORS = hyperparams['num_factors']
NUM_ECO_LETTERS = hyperparams['num_eco_letters']
NUM_ECO_NUMBERS = hyperparams['num_eco_numbers']
ECO_EMBED_DIM = hyperparams['eco_embed_dim']

print(f"✓ Loaded hyperparameters")
print(f"   Num players: {NUM_PLAYERS:,}")
print(f"   Num openings: {NUM_OPENINGS:,}")
print(f"   Num factors: {NUM_FACTORS}")
print(f"   ECO letters: {NUM_ECO_LETTERS}, ECO numbers: {NUM_ECO_NUMBERS}")
print(f"   ECO embedding dim: {ECO_EMBED_DIM}")


Loading model hyperparameters...
✓ Loaded hyperparameters
   Num players: 48,551
   Num openings: 2,728
   Num factors: 40
   ECO letters: 5, ECO numbers: 100
   ECO embedding dim: 4


In [7]:
# Load opening mappings for display
print("\nLoading opening mappings...")

opening_mappings_path = MODEL_ARTIFACTS_DIR / "opening_mappings.csv"
opening_mappings_df = pd.read_csv(opening_mappings_path)

print(f"✓ Loaded opening mappings")
print(f"   Total openings: {len(opening_mappings_df):,}")
print(f"   Columns: {list(opening_mappings_df.columns)}")
print(f"\n   Sample openings:")
print(opening_mappings_df[['training_id', 'eco', 'name']].head(3).to_string(index=False))


Loading opening mappings...
✓ Loaded opening mappings
   Total openings: 2,728
   Columns: ['db_id', 'eco', 'name', 'training_id']

   Sample openings:
 training_id eco                       name
           0 A00               Amar Opening
           1 A00 Amar Opening: Paris Gambit
           2 A00        Anderssen's Opening


In [8]:
# Load side information (ECO encodings) for ALL openings
print("\nLoading and constructing side information...")

# Load ECO encodings
eco_encodings_path = MODEL_ARTIFACTS_DIR / "eco_encodings.json"
with open(eco_encodings_path, 'r') as f:
    eco_encodings = json.load(f)

eco_letter_map = eco_encodings['eco_letter_to_int']
eco_number_map = eco_encodings['eco_number_to_int']

# Construct side information from opening_mappings_df
# Extract ECO letter and number for each opening
eco_letters = []
eco_numbers = []

for _, row in opening_mappings_df.iterrows():
    eco_code = row['eco']  # e.g., "B02", "C45"
    
    # Parse ECO letter (first character)
    eco_letter = eco_code[0]
    eco_letter_cat = eco_letter_map[eco_letter]
    
    # Parse ECO number (remaining characters)
    eco_number = eco_code[1:]  # e.g., "02", "45"
    eco_number_cat = eco_number_map[eco_number]
    
    eco_letters.append(eco_letter_cat)
    eco_numbers.append(eco_number_cat)

# Convert to tensors for efficient batch processing
# These are sorted by training_id (0, 1, 2, ..., N-1)
all_opening_ids = torch.tensor(opening_mappings_df['training_id'].values, dtype=torch.long)
all_eco_letter_cats = torch.tensor(eco_letters, dtype=torch.long)
all_eco_number_cats = torch.tensor(eco_numbers, dtype=torch.long)

print(f"✓ Constructed side information for {len(all_opening_ids)} openings")
print(f"   All opening IDs shape: {all_opening_ids.shape}")
print(f"   All ECO letter cats shape: {all_eco_letter_cats.shape}")
print(f"   All ECO number cats shape: {all_eco_number_cats.shape}")
print(f"   ECO letter range: [{all_eco_letter_cats.min()}, {all_eco_letter_cats.max()}]")
print(f"   ECO number range: [{all_eco_number_cats.min()}, {all_eco_number_cats.max()}]")


Loading and constructing side information...
✓ Constructed side information for 2728 openings
   All opening IDs shape: torch.Size([2728])
   All ECO letter cats shape: torch.Size([2728])
   All ECO number cats shape: torch.Size([2728])
   ECO letter range: [0, 4]
   ECO number range: [0, 99]
✓ Constructed side information for 2728 openings
   All opening IDs shape: torch.Size([2728])
   All ECO letter cats shape: torch.Size([2728])
   All ECO number cats shape: torch.Size([2728])
   ECO letter range: [0, 4]
   ECO number range: [0, 99]


In [13]:
# Load player ratings (needed for model buffers)
print("\nLoading player ratings...")

# Load player mappings to get ratings
player_mappings_path = MODEL_ARTIFACTS_DIR / "player_mappings.csv"
player_mappings_df = pd.read_csv(player_mappings_path)

# Create player ratings tensor (sorted by training_id)
player_mappings_df = player_mappings_df.sort_values('training_id')
player_ratings_tensor = torch.tensor(player_mappings_df['rating'].values, dtype=torch.float32)

# Load rating normalization parameters
rating_norm_path = MODEL_ARTIFACTS_DIR / "rating_normalization.json"
with open(rating_norm_path, 'r') as f:
    rating_norm = json.load(f)

# Normalize ratings (z-score)
player_ratings_tensor = (player_ratings_tensor - rating_norm['rating_mean']) / rating_norm['rating_std']

print(f"✓ Loaded and normalized player ratings")
print(f"   Shape: {player_ratings_tensor.shape}")
print(f"   Range: [{player_ratings_tensor.min():.4f}, {player_ratings_tensor.max():.4f}]")
print(f"   Mean: {player_ratings_tensor.mean():.4f}")
print(f"   Std: {player_ratings_tensor.std():.4f}")


Loading player ratings...
✓ Loaded and normalized player ratings
   Shape: torch.Size([48551])
   Range: [-2.2687, 4.2454]
   Mean: -0.0000
   Std: 1.0000


---

# PART 3: DEFINE MODEL ARCHITECTURE

Define the `ChessOpeningRecommender` model class.

This is copied from notebook 28 (training pipeline) to ensure exact architecture match.

In [15]:
class ChessOpeningRecommender(nn.Module):
    """
    Matrix Factorization model for chess opening recommendations.
    
    The model learns latent factors for players and openings, incorporating
    side information:
    - Player ratings (normalized)
    - Opening ECO codes (letter and number as categorical features)
    
    Architecture:
    - Player embedding: learnable latent factors
    - Opening embedding: learnable latent factors
    - Player rating: fixed side information
    - ECO letter/number: categorical embeddings
    
    Prediction: dot product of player and opening representations + biases
    """
    
    def __init__(
        self,
        num_players: int,
        num_openings: int,
        num_factors: int,
        player_ratings: torch.Tensor,
        opening_eco_letters: torch.Tensor,
        opening_eco_numbers: torch.Tensor,
        num_eco_letters: int,
        num_eco_numbers: int,
        eco_embed_dim: int = 4
    ):
        """
        Initialize the recommendation model.
        
        Args:
            num_players: Total number of unique players
            num_openings: Total number of unique openings
            num_factors: Dimensionality of latent factor embeddings
            player_ratings: Z-score normalized ratings for all players (shape: [num_players])
            opening_eco_letters: ECO letter categories for all openings (shape: [num_openings])
            opening_eco_numbers: ECO number categories for all openings (shape: [num_openings])
            num_eco_letters: Number of unique ECO letter categories
            num_eco_numbers: Number of unique ECO number categories
            eco_embed_dim: Dimensionality of ECO categorical embeddings (default: 4)
        """
        super(ChessOpeningRecommender, self).__init__()
        
        # Store configuration
        self.num_players = num_players
        self.num_openings = num_openings
        self.num_factors = num_factors
        self.eco_embed_dim = eco_embed_dim
        
        # ========================================
        # Player Components
        # ========================================
        
        self.player_factors = nn.Embedding(num_players, num_factors)
        self.player_biases = nn.Embedding(num_players, 1)
        
        # Player ratings (fixed side information - registered as buffer)
        self.register_buffer('player_ratings', player_ratings)
        
        # ========================================
        # Opening Components
        # ========================================
        
        self.opening_factors = nn.Embedding(num_openings, num_factors)
        self.opening_biases = nn.Embedding(num_openings, 1)
        
        # ECO letter and number (fixed side information - registered as buffers)
        self.register_buffer('opening_eco_letters', opening_eco_letters)
        self.register_buffer('opening_eco_numbers', opening_eco_numbers)
        
        # ECO embeddings (learnable)
        self.eco_letter_embedding = nn.Embedding(num_eco_letters, eco_embed_dim)
        self.eco_number_embedding = nn.Embedding(num_eco_numbers, eco_embed_dim)
        
        # ========================================
        # Combination Layers
        # ========================================
        
        # Combine player latent factors with rating
        # Input: [num_factors + 1] → Output: [num_factors]
        self.player_combiner = nn.Linear(num_factors + 1, num_factors)
        
        # Combine opening latent factors with ECO embeddings
        # Input: [num_factors + 2*eco_embed_dim] → Output: [num_factors]
        self.opening_combiner = nn.Linear(num_factors + 2 * eco_embed_dim, num_factors)
        
        # ========================================
        # Global Bias
        # ========================================
        
        # Global bias term (learnable scalar)
        self.global_bias = nn.Parameter(torch.zeros(1))
        
        # ========================================
        # Initialize Weights
        # ========================================
        
        # Initialize embeddings with small random values
        nn.init.normal_(self.player_factors.weight, mean=0, std=0.01)
        nn.init.normal_(self.opening_factors.weight, mean=0, std=0.01)
        nn.init.normal_(self.eco_letter_embedding.weight, mean=0, std=0.01)
        nn.init.normal_(self.eco_number_embedding.weight, mean=0, std=0.01)
        
        # Initialize biases to zero
        nn.init.zeros_(self.player_biases.weight)
        nn.init.zeros_(self.opening_biases.weight)
        
        # Initialize linear layers with Xavier initialization
        nn.init.xavier_uniform_(self.player_combiner.weight)
        nn.init.zeros_(self.player_combiner.bias)
        nn.init.xavier_uniform_(self.opening_combiner.weight)
        nn.init.zeros_(self.opening_combiner.bias)
    
    def forward(self, player_ids: torch.Tensor, opening_ids: torch.Tensor) -> torch.Tensor:
        """
        Forward pass: predict player-opening scores.
        
        Args:
            player_ids: Player IDs (shape: [batch_size])
            opening_ids: Opening IDs (shape: [batch_size])
            
        Returns:
            Predicted scores (shape: [batch_size])
        """
        # ========================================
        # Get Player Representation
        # ========================================
        
        # Get player latent factors [batch_size, num_factors]
        player_embed = self.player_factors(player_ids)
        
        # Get player ratings [batch_size, 1]
        player_rating = self.player_ratings[player_ids].unsqueeze(1)
        
        # Concatenate player factors with rating [batch_size, num_factors + 1]
        player_concat = torch.cat([player_embed, player_rating], dim=1)
        
        # Combine into final player representation [batch_size, num_factors]
        player_repr = self.player_combiner(player_concat)
        
        # Get player bias [batch_size]
        player_bias = self.player_biases(player_ids).squeeze()
        
        # ========================================
        # Get Opening Representation
        # ========================================
        
        # Get opening latent factors [batch_size, num_factors]
        opening_embed = self.opening_factors(opening_ids)
        
        # Get ECO embeddings
        eco_letters = self.opening_eco_letters[opening_ids]  # [batch_size]
        eco_numbers = self.opening_eco_numbers[opening_ids]  # [batch_size]
        
        eco_letter_embed = self.eco_letter_embedding(eco_letters)  # [batch_size, eco_embed_dim]
        eco_number_embed = self.eco_number_embedding(eco_numbers)  # [batch_size, eco_embed_dim]
        
        # Concatenate opening factors with ECO embeddings
        # [batch_size, num_factors + 2*eco_embed_dim]
        opening_concat = torch.cat([opening_embed, eco_letter_embed, eco_number_embed], dim=1)
        
        # Combine into final opening representation [batch_size, num_factors]
        opening_repr = self.opening_combiner(opening_concat)
        
        # Get opening bias [batch_size]
        opening_bias = self.opening_biases(opening_ids).squeeze()
        
        # ========================================
        # Compute Prediction
        # ========================================
        
        # Dot product of player and opening representations [batch_size]
        interaction = (player_repr * opening_repr).sum(dim=1)
        
        # Add biases and global bias
        prediction = interaction + player_bias + opening_bias + self.global_bias
        
        # Apply sigmoid to constrain output to [0, 1] range
        prediction = torch.sigmoid(prediction)
        
        return prediction

print("✓ ChessOpeningRecommender model class defined")

✓ ChessOpeningRecommender model class defined


---

# PART 4: LOAD TRAINED MODEL

Instantiate the model with correct hyperparameters and load trained weights.

In [16]:
# Instantiate model
print("Instantiating model...")

model = ChessOpeningRecommender(
    num_players=NUM_PLAYERS,
    num_openings=NUM_OPENINGS,
    num_factors=NUM_FACTORS,
    player_ratings=player_ratings_tensor,
    opening_eco_letters=all_eco_letter_cats,
    opening_eco_numbers=all_eco_number_cats,
    num_eco_letters=NUM_ECO_LETTERS,
    num_eco_numbers=NUM_ECO_NUMBERS,
    eco_embed_dim=ECO_EMBED_DIM
)

print(f"✓ Model instantiated")
print(f"   Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"   Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

Instantiating model...
✓ Model instantiated
   Total parameters: 2,106,500
   Trainable parameters: 2,106,500


In [17]:
# Load trained weights
print("\nLoading trained weights...")

model_path = MODEL_ARTIFACTS_DIR / "best_model.pt"
model.load_state_dict(torch.load(model_path, map_location=DEVICE))
model.to(DEVICE)
model.eval()  # Set to evaluation mode (disables dropout, etc.)

print(f"✓ Loaded trained weights from: {model_path.name}")
print(f"   Model is on device: {next(model.parameters()).device}")
print(f"   Model is in eval mode: {not model.training}")


Loading trained weights...
✓ Loaded trained weights from: best_model.pt
   Model is on device: cpu
   Model is in eval mode: True


---

# PART 5: DEFINE INFERENCE FUNCTIONS

Define pure functions for inference pipeline.

**Design Principle**: Each function is small, testable, and reusable.

In [18]:
def convert_to_tensors(
    model_input: ModelInput,
    device: torch.device = torch.device('cpu')
) -> Dict[str, torch.Tensor]:
    """
    Convert ModelInput to PyTorch tensors for inference.
    
    Args:
        model_input: Processed player data from notebook 29
        device: Device to place tensors on (cpu or cuda)
    
    Returns:
        Dictionary of tensors:
            - player_ids: [batch_size] - player indices (or zeros for fold-in)
            - opening_ids: [batch_size] - opening indices
    
    Note: ECO features and player ratings are accessed from model buffers,
    not passed as forward() arguments.
    """
    batch_size = len(model_input)
    
    # Handle fold-in users (training_player_id is None)
    # Use player_id=0 (first player in training set) as a default
    if model_input.training_player_id is None:
        player_ids = torch.zeros(batch_size, dtype=torch.long, device=device)
    else:
        player_ids = torch.full(
            (batch_size,), 
            model_input.training_player_id, 
            dtype=torch.long, 
            device=device
        )
    
    # Convert opening IDs to tensors
    opening_ids = torch.tensor(model_input.opening_ids, dtype=torch.long, device=device)
    
    return {
        'player_ids': player_ids,
        'opening_ids': opening_ids
    }

print("✓ convert_to_tensors() defined")

✓ convert_to_tensors() defined


In [28]:
@torch.no_grad()  # Disable gradient computation for inference
def generate_predictions(
    model: nn.Module,
    tensors: Dict[str, torch.Tensor]
) -> np.ndarray:
    """
    Generate predictions for the given tensors.
    
    Args:
        model: Trained ChessOpeningRecommender model (in eval mode)
        tensors: Dictionary of input tensors from convert_to_tensors()
    
    Returns:
        predictions: [batch_size] numpy array of predicted scores (0-1)
    """
    # Run forward pass (model accesses ECO features and ratings from buffers)
    predictions = model(
        player_ids=tensors['player_ids'],
        opening_ids=tensors['opening_ids']
    )
    
    # Convert to numpy - handle both tensor and scalar cases
    predictions_np = predictions.detach().cpu()
    if predictions_np.dim() > 0:
        predictions_np = predictions_np.squeeze()
    return predictions_np.numpy()

print("✓ generate_predictions() defined")

✓ generate_predictions() defined


In [29]:
@torch.no_grad()
def predict_all_openings(
    model: nn.Module,
    rating_z: float,
    all_opening_ids: torch.Tensor,
    player_id: Optional[int] = None,
    device: torch.device = torch.device('cpu'),
    batch_size: int = 512
) -> np.ndarray:
    """
    Predict performance scores for ALL valid openings (not just played ones).
    
    This is the key function for generating recommendations - it scores every
    opening in the training set, allowing us to recommend new openings the
    player hasn't tried yet.
    
    Args:
        model: Trained ChessOpeningRecommender model (in eval mode)
        rating_z: Player's z-score normalized rating (not used in forward pass,
                  but kept for documentation/future use)
        all_opening_ids: [num_openings] - all valid opening IDs
        player_id: Player's training ID (None for fold-in users)
        device: Device to run inference on
        batch_size: Batch size for inference (to avoid memory issues)
    
    Returns:
        predictions: [num_openings] numpy array of predicted scores
    
    Note: Model accesses ECO features and player ratings from internal buffers,
    not from function arguments. The rating_z parameter is kept for reference
    but not passed to forward().
    """
    num_openings = len(all_opening_ids)
    all_predictions = []
    
    # Process in batches to avoid memory issues
    for start_idx in range(0, num_openings, batch_size):
        end_idx = min(start_idx + batch_size, num_openings)
        batch_len = end_idx - start_idx
        
        # Get batch of openings
        batch_opening_ids = all_opening_ids[start_idx:end_idx].to(device)
        
        # Create player tensors (same for all openings in batch)
        if player_id is None:
            batch_player_ids = torch.zeros(batch_len, dtype=torch.long, device=device)
        else:
            batch_player_ids = torch.full((batch_len,), player_id, dtype=torch.long, device=device)
        
        # Generate predictions for batch
        # Model accesses ECO features and ratings from buffers
        batch_predictions = model(
            player_ids=batch_player_ids,
            opening_ids=batch_opening_ids
        )
        
        # Convert to numpy
        batch_predictions_np = batch_predictions.detach().cpu()
        if batch_predictions_np.dim() > 0:
            batch_predictions_np = batch_predictions_np.squeeze()
        all_predictions.append(batch_predictions_np.numpy())
    
    # Concatenate all batches
    return np.concatenate(all_predictions)

print("✓ predict_all_openings() defined")

✓ predict_all_openings() defined


In [21]:
def rank_recommendations(
    predictions: np.ndarray,
    opening_mappings_df: pd.DataFrame,
    played_opening_ids: Optional[np.ndarray] = None,
    top_n: int = 10
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Rank and filter opening recommendations.
    
    Args:
        predictions: [num_openings] - predicted scores for all openings
        opening_mappings_df: DataFrame with opening metadata
        played_opening_ids: Array of opening IDs player has already played
        top_n: Number of top recommendations to return
    
    Returns:
        Tuple of (played_df, unplayed_df):
            - played_df: Predictions for openings player has played (sorted by pred)
            - unplayed_df: Top N recommendations for unplayed openings
    """
    # Create results DataFrame
    results_df = opening_mappings_df.copy()
    results_df['predicted_score'] = predictions
    
    # Split into played and unplayed
    if played_opening_ids is not None:
        played_mask = results_df['training_id'].isin(played_opening_ids)
        played_df = results_df[played_mask].sort_values('predicted_score', ascending=False)
        unplayed_df = results_df[~played_mask].sort_values('predicted_score', ascending=False)
    else:
        played_df = pd.DataFrame()  # Empty if no played openings provided
        unplayed_df = results_df.sort_values('predicted_score', ascending=False)
    
    # Take top N unplayed
    unplayed_df = unplayed_df.head(top_n)
    
    return played_df, unplayed_df

print("✓ rank_recommendations() defined")

✓ rank_recommendations() defined


---

# PART 6: RUN INFERENCE

Apply the functions to our loaded data.

In [23]:
# Step 6.1: Convert player data to tensors
print("Converting player data to tensors...")

input_tensors = convert_to_tensors(model_input, device=DEVICE)

print(f"✓ Tensors created")
print(f"   player_ids: {input_tensors['player_ids'].shape} on {input_tensors['player_ids'].device}")
print(f"   opening_ids: {input_tensors['opening_ids'].shape}")

Converting player data to tensors...
✓ Tensors created
   player_ids: torch.Size([94]) on cpu
   opening_ids: torch.Size([94])


In [30]:
# Step 6.2: Generate predictions for played openings
print("\nGenerating predictions for played openings...")

played_predictions = generate_predictions(model, input_tensors)

print(f"✓ Predictions generated")
print(f"   Shape: {played_predictions.shape}")
print(f"   Range: [{played_predictions.min():.4f}, {played_predictions.max():.4f}]")
print(f"   Mean: {played_predictions.mean():.4f}")


Generating predictions for played openings...


RuntimeError: Numpy is not available

In [ ]:
# Step 6.3: Predict scores for ALL openings (including unplayed)
print("\nPredicting scores for ALL valid openings...")

all_predictions = predict_all_openings(
    model=model,
    rating_z=model_input.rating_z,
    all_opening_ids=all_opening_ids,
    player_id=model_input.training_player_id,
    device=DEVICE,
    batch_size=512
)

print(f"✓ Predictions generated for all openings")
print(f"   Total openings scored: {len(all_predictions):,}")
print(f"   Range: [{all_predictions.min():.4f}, {all_predictions.max():.4f}]")
print(f"   Mean: {all_predictions.mean():.4f}")

In [ ]:
# Step 6.4: Rank and filter recommendations
print("\nRanking recommendations...")

played_df, unplayed_df = rank_recommendations(
    predictions=all_predictions,
    opening_mappings_df=opening_mappings_df,
    played_opening_ids=model_input.opening_ids,
    top_n=TOP_N_RECOMMENDATIONS
)

print(f"✓ Recommendations ranked")
print(f"   Played openings: {len(played_df)}")
print(f"   Unplayed openings (total): {len(opening_mappings_df) - len(played_df)}")
print(f"   Top recommendations: {len(unplayed_df)}")

---

# PART 7: ANALYZE RESULTS

Display and analyze the predictions.

In [ ]:
# Compare predictions vs actuals for played openings
print("═" * 80)
print("PREDICTION ANALYSIS: PLAYED OPENINGS")
print("═" * 80)

# Add actual scores to played_df
# Match by training_id to ensure correct alignment
played_df_with_actuals = played_df.copy()
played_df_with_actuals = played_df_with_actuals.set_index('training_id')

# Create a mapping from training_id to actual score
actual_scores_map = dict(zip(model_input.opening_ids, model_input.scores))
played_df_with_actuals['actual_score'] = played_df_with_actuals.index.map(actual_scores_map)
played_df_with_actuals = played_df_with_actuals.reset_index()

# Calculate error metrics
errors = played_df_with_actuals['predicted_score'] - played_df_with_actuals['actual_score']
mae = np.abs(errors).mean()
rmse = np.sqrt((errors ** 2).mean())
mse = (errors ** 2).mean()

print(f"\nError Metrics:")
print(f"   MAE:  {mae:.4f}")
print(f"   RMSE: {rmse:.4f}")
print(f"   MSE:  {mse:.4f}")

# Show top predicted vs actual
print(f"\nTop 10 Played Openings by Predicted Score:")
display_cols = ['eco', 'name', 'predicted_score', 'actual_score']
print(played_df_with_actuals[display_cols].head(10).to_string(index=False))

print("\n" + "═" * 80)

In [ ]:
# Display top recommendations (unplayed openings)
print("═" * 80)
print(f"TOP {TOP_N_RECOMMENDATIONS} OPENING RECOMMENDATIONS (UNPLAYED)")
print("═" * 80)

print(f"\nThese are the {COLOR_NAME} openings the model predicts you will perform best with:")
print(f"(You haven't played these openings yet in the training data)\n")

# Format recommendations for display
recommendations = unplayed_df[['eco', 'name', 'predicted_score']].copy()
recommendations['rank'] = range(1, len(recommendations) + 1)
recommendations = recommendations[['rank', 'eco', 'name', 'predicted_score']]

# Pretty print
for _, row in recommendations.iterrows():
    print(f"{row['rank']:2d}. {row['eco']:<6} {row['name']:<60} ({row['predicted_score']:.4f})")

print("\n" + "═" * 80)

In [ ]:
# Summary statistics
print("═" * 80)
print("INFERENCE SUMMARY")
print("═" * 80)

print(f"\nPlayer Information:")
print(f"   Training Player ID: {model_input.training_player_id} {'(fold-in user)' if model_input.training_player_id is None else ''}")
print(f"   Rating Z-score: {model_input.rating_z:.4f}")
print(f"   Color: {COLOR_NAME}")

print(f"\nOpenings Analysis:")
print(f"   Total valid openings in training: {len(opening_mappings_df):,}")
print(f"   Openings played by player: {len(model_input)}")
print(f"   Openings NOT yet played: {len(opening_mappings_df) - len(model_input):,}")

print(f"\nPrediction Statistics (All Openings):")
print(f"   Min predicted score: {all_predictions.min():.4f}")
print(f"   Max predicted score: {all_predictions.max():.4f}")
print(f"   Mean predicted score: {all_predictions.mean():.4f}")
print(f"   Median predicted score: {np.median(all_predictions):.4f}")

if len(played_df_with_actuals) > 0:
    print(f"\nPrediction Quality (Played Openings):")
    print(f"   MAE: {mae:.4f}")
    print(f"   RMSE: {rmse:.4f}")
    print(f"   Correlation: {played_df_with_actuals['predicted_score'].corr(played_df_with_actuals['actual_score']):.4f}")

print(f"\nRecommendations:")
print(f"   Top {TOP_N_RECOMMENDATIONS} unplayed openings have been identified")
print(f"   Predicted score range for top {TOP_N_RECOMMENDATIONS}: [{unplayed_df['predicted_score'].min():.4f}, {unplayed_df['predicted_score'].max():.4f}]")

print("\n" + "═" * 80)
print("✓ INFERENCE COMPLETE")
print("═" * 80)

---

## Next Steps

1. **Production Deployment**: Adapt functions above for HuggingFace Spaces
2. **API Integration**: Replace `pickle.load()` with Lichess API calls
3. **Batch Processing**: Process multiple users in parallel
4. **Caching**: Cache model and artifacts in memory for fast inference
5. **Monitoring**: Add logging and performance metrics

---

## Performance Notes

- **Model Loading**: ~2-4 seconds (one-time at startup)
- **Per-User Inference**: ~1-2 seconds
  - Data transformation: ~0.5s
  - Model forward pass: ~0.5s
  - Post-processing: ~0.2s
- **Total**: Well within HuggingFace Spaces 60-second timeout ✓